In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv('winequality-red.csv')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [8]:
fix_data = data.copy()
fix_data['fixed acidity'] /= fix_data['fixed acidity'].max()
fix_data['residual sugar'] /= fix_data['residual sugar'].max()
fix_data['free sulfur dioxide'] /= fix_data['free sulfur dioxide'].max()
fix_data['total sulfur dioxide'] /= fix_data['total sulfur dioxide'].max()
fix_data['pH'] /= fix_data['pH'].max()
fix_data['alcohol'] /= fix_data['alcohol'].max()

In [9]:
y = fix_data['quality'].values
X = fix_data.drop(['quality'], axis=1).values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [11]:
y_train = np.resize(y_train, (1279, 1))
y_test = np.resize(y_test, (320, 1))

# Start build NN

In [12]:
layer_1 = np.random.random((11, 5)) * 2 - 1
layer_2 = np.random.random((5, 1)) * 2 - 1

In [13]:
def leaky_relu(layer, alpha=0.1):
    layer[layer < 0] *= alpha
    return layer

In [14]:
def predict(x):
    x = leaky_relu(x @ layer_1)
    x = leaky_relu(x @ layer_2)
    return x

In [15]:
def fit(epoch=1000, X=X_train, Y=y_train, batch_size=250, lr = 1e-3, layer_2=layer_2, layer_1=layer_1):
    for _ in range(epoch):

        i, n = 0, 1

        while X[i: n * batch_size].size != 0:

            inp = X_train[i:n * 250]
            y = np.resize(y_train[i:n * 250], (inp.shape[0], 1))
            i = n * 250
            n += 1

            x_1 = leaky_relu(inp @ layer_1)
            out = leaky_relu(x_1 @ layer_2)

            error_grad = 2 * (y - out)

            layer_1 += (inp.T @ ((error_grad @ layer_2.T) * x_1)) * 1e-6
            layer_2 += (x_1.T @ error_grad) * 1e-6

In [16]:
fit(1000)

In [18]:
sum(np.round(predict(X_test)) == y_test) / len(y_test), sum(np.round(predict(X_train)) == y_train) / 1279

# nice

(array([0.4875]), array([0.45582486]))